Open the bash container:

In [ ]:
img_path=../0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /dodrio,/readonly/dodrio,/tmp $img_path bash

In [ ]:
img_path=../0_resources/vsn_cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /lustre1,/staging,/data,${VSC_SCRATCH},${VSC_SCRATCH}/tmp:/tmp,${HOME}/.nextflow/assets/ $img_path bash

Generate fragments:

In [4]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8

subset=
for file in public_preprocessing_output/data/reports/barcard/*$subset*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.tomerge.tsv}
    # echo $sample
    jaccardfile=$file
    infile=public_preprocessing_output/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=public_preprocessing_output/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz

    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    elif [ -f "$outfile2" ]; then
        echo "$outfile2 exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
    fi
done

full_preprocessing_out/data/reports/barcard/BIO_ddseq_1.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_1.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BIO_ddseq_2.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_2.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BIO_ddseq_3.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_3.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BIO_ddseq_4.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BIO_ddseq_4.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BRO_mtscatac_1.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_1.FULL.fragments.tsv.gz exists, skipping
full_preprocessing_out/data/reports/barcard/BRO_mtscatac_2.FULL.tomerge.tsv
full_preprocessing_out/data/fragments/BRO_mtscatac_2.FULL.fragments.tsv.gz exists, skipping
full_preprocessi

Then bgzip the output:

In [2]:
module load HTSlib
subset=
for file in public_preprocessing_output/data/fragments/*$subset*.fragments.tsv
do
    echo $file
    if [ -f "$file.gz" ]; then
        echo exists
    else
        bgzip -@ 8 $file &
    fi
done

full_preprocessing_out/data/fragments/OHS_s3atac_1.FULL.fragments.tsv
exists
full_preprocessing_out/data/fragments/OHS_s3atac_2.FULL.fragments.tsv
exists


and tabix index

In [ ]:
module load HTSlib
subset=
for file in public_preprocessing_output/data/fragments/*$subset*.fragments.tsv.gz
do
    echo $file
    if [ -f "$file.gz.tbi" ]; then
        echo exists
    else
        tabix -p bed $file &
    fi
done